In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
import torch
import psutil
import cpuinfo

# Experiment Tracking
import wandb

# Sklearn Metrics
from sklearn.metrics import (
    accuracy_score, 
    precision_recall_fscore_support, 
    matthews_corrcoef, 
    confusion_matrix
)

# Hugging Face Datasets
from datasets import Dataset, load_from_disk

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)

# PEFT (Parameter Efficient Fine-Tuning)
from peft import (
    LoraConfig, 
    get_peft_model, 
    TaskType,
    PeftModel
)

In [ ]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
# import multiprocessing as mp
# mp.set_start_method('spawn', force=True)

# GPU & CPU Specs

In [12]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))

Torch version: 2.5.1+cu124
CUDA available: True
GPU name: Tesla T4


In [ ]:
# Get CPU model and cores
cpu_info = cpuinfo.get_cpu_info()
print(f"CPU Model: {cpu_info['brand_raw']}")
print(f"Cores: {psutil.cpu_count(logical=False)} Physical / {psutil.cpu_count(logical=True)} Logical")

# Get clock speed
print(f"Max Frequency: {psutil.cpu_freq().max:.2f} MHz")

# Check RAM
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

CPU Model: Intel(R) Xeon(R) CPU @ 2.00GHz
Cores: 2 Physical / 4 Logical
Max Frequency: 0.00 MHz
RAM: 31.4 GB


# Data Loading

In [ ]:
# Load from read-only input
original_dataset = load_from_disk("/kaggle/input/merged/tokenized_merged_wo_emojis")

working_dir = "/kaggle/working/merged/tokenized_merged_wo_emojis"
# Save a copy to a writable location
original_dataset.save_to_disk(working_dir)

# Load the writable copy
tokenized_dataset = load_from_disk(working_dir, keep_in_memory=True)

Saving the dataset (0/1 shards):   0%|          | 0/494172 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/30000 [00:00<?, ? examples/s]

In [15]:
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"], device="cuda")

In [16]:
print(tokenized_dataset["train"].column_names)

['labels', 'input_ids', 'attention_mask']


# Sweep Config

In [ ]:
sweep_config = {
    'name': 'Fourth Sweep',
    'method': 'bayes',  # Bayesian optimization
    'metric': {
        'name': 'test_matthews_corrcoef',
        'goal': 'maximize'
    },
    'early_terminate': {
        'type': 'hyperband',  # Revert to original hyperband termination
        'min_iter': 3         # Keep original minimum iterations
    },
    'parameters': {
        'target_modules': {
            'values': [
                ["query", "key"],
                ["query", "key", "value"],
                ["query", "key", "value", "attention.output.dense"]
            ]
        },
        'lora_r': {
            'values': [16, 32, 64]
        },
        'lora_alpha': {
            'values': [4, 8, 12, 16, 24, 32, 48]
        },
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="bertweet-lora-bayes-v2")

In [ ]:
def compute_metrics(pred):
    logits, labels = pred.predictions, pred.label_ids
    
    # Optional sanity check: make sure logits are not probabilities
    if logits.shape[1] == 2:  # Binary classification logits
        probs = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
    else:
        raise ValueError("Unexpected logits shape: make sure model outputs raw logits with 2 classes.")
    
    preds = (probs > 0.65).astype(int)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(labels, preds, average=None, zero_division=0)

    acc = accuracy_score(labels, preds)
    mcc = matthews_corrcoef(labels, preds)

    wandb.log({
        "confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=labels,
            preds=preds,
            class_names=["Non-Hate", "Hate"]
        ),
        "threshold_used": 0.65  # <-- Log the threshold too!
    })

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'matthews_corrcoef': mcc,
        'precision_class_0': precision_per_class[0],
        'precision_class_1': precision_per_class[1],
        'recall_class_0': recall_per_class[0],
        'recall_class_1': recall_per_class[1],
        'f1_class_0': f1_per_class[0],
        'f1_class_1': f1_per_class[1],
    }

In [22]:
# ============================
# 📦 Dataset & Batching Config
# ============================
data_config = {
    "per_device_train_batch_size": 512,
    "per_device_eval_batch_size": 256,
    "dataloader_pin_memory": False,
    "label_names": ["labels"]
}

# ========================
# ⚙️ Training Runtime Setup
# ========================
runtime_config = {
    "num_train_epochs": 1,
    "gradient_accumulation_steps": 2,
    "fp16": True,
    "gradient_checkpointing": True
}

# =============================
# 🔧 Optimization Configuration
# =============================
optimizer_config = {
    "optim": "adamw_torch_fused",
    "lr_scheduler_type": "cosine",
    "learning_rate": 2e-3,
    "max_grad_norm": 1.0,
    "warmup_ratio": 0.1,
    "weight_decay": 0.01
}

# ==============================
# 📊 Logging & Saving Strategy
# ==============================
logging_config = {
    "report_to": "wandb",
    "logging_steps": 200,
    "eval_strategy": "steps",
    "eval_steps": 200,
    "save_strategy": "steps",
    "save_steps": 200,
    "load_best_model_at_end": True,
    "metric_for_best_model": "matthews_corrcoef",
    "greater_is_better": True
}

# ===============================
# 🧬 Final Training Configuration
# ===============================
training_kwargs = {
    **data_config,
    **runtime_config,
    **optimizer_config,
    **logging_config
}

# Training

In [24]:
model_name = 'vinai/bertweet-base'

model_kwargs = {
    "num_labels": 2,
    "device_map": "cuda",
    "low_cpu_mem_usage": True,
    # "quantization_config": bnb_config
}

In [ ]:
def train(config=None):
    # Initialize W&B run
    with wandb.init(config=config): 
        # Get hyperparameters from W&B
        config = wandb.config
        
        # Load model with all kwargs
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            **model_kwargs
        )
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        
        # Setup LoRA
        peft_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=config.lora_r,
            lora_alpha= config.lora_alpha,
            lora_dropout=0.1,
            bias="all",
            target_modules= config.target_modules,
            modules_to_save=["classifier"]
        )
                
        model = get_peft_model(model, peft_config)

        # model = torch.compile(model)
                
        # Fix label mappings
        model.config.id2label = {0: "Non-Hate", 1: "Hate"}
        model.config.label2id = {"Non-Hate": 0, "Hate": 1}
        
        # Calculate trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())
        trainable_ratio = 100 * trainable_params / total_params
        
        model.print_trainable_parameters()

    
        training_args = TrainingArguments(
        **training_kwargs
    )
    
        # Initialize trainer
    trainer = Trainer(
            model=model, args=training_args,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["validation"],
            compute_metrics=compute_metrics
        )
    
    # Train and evaluate
    trainer.train()
    test_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])

    results_to_log = {
        # Training metrics
        "trainable_params": trainable_params,
        "trainable_ratio": trainable_ratio,
        
        # Evaluation metrics (prefixed for clarity)
        "test_matthews_corrcoef": test_results["eval_matthews_corrcoef"],
        "test/accuracy": test_results["eval_accuracy"],
        "test/f1": test_results["eval_f1"],
        
        # Hyperparameters
        "hp/lora_r": config.lora_r,
        "hp/lora_alpha": config.lora_alpha,
        "hp/lora_ratio": config.lora_alpha/config.lora_r,
    }
    
    wandb.log(results_to_log)

In [26]:
# def run_agent(gpu_id):
#     os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
#     wandb.agent(sweep_id, function=train, count=16)

# if __name__ == "__main__":
#     processes = []
#     for gpu in [0, 1]:
#         p = mp.Process(target=run_agent, args=(gpu,))
#         p.start()
#         processes.append(p)

#     for p in processes:
#         p.join()

In [ ]:
# Run the sweep
wandb.agent(sweep_id, function=train, count=24)